In [2]:
##################################################################################
#                          Dashboard Application                                 #
#                                                                                #
# Description: A dashboard application that works with existing data to identify #
# and categorize available dogs that are good candidates for search-and-rescue   #
# training.                                                                      #
# Alyssa Fasciano                                                                #
# Southern New Hampshire University                                              #
# CS-499: Computer Science Capstone                                              #
# March/April 2023                                                               #
##################################################################################
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Username and password for user authetication
username = "aacuser"
password = "abcdef"
shelter = AnimalShelter(username, password)


# Read all method supports return of cursor object 
df = pd.DataFrame.from_records(shelter.readAll({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Dashboard Application')

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Dashboard Application'))),
    html.Hr(),
    # Cat/Dog filtering options 
    html.Div(className='row',
            style={'display' : 'flex'},
            children=[
                html.Button(id='submit-button-one', n_clicks=0, children='Cats'),
                html.Button(id='submit-button-two', n_clicks=0, children='Dogs')
            ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
# Additional features for the interactive data table to make it user-friendly
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
# Sets up dashboard with pie chart and geolocation chart side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.H4('Alyssa Fasciano, CS-499')
])

#############################################
# Interaction Between Components / Controller
#############################################

# Callback for cat/dog filtering options
@app.callback(Output('datatable-id', "data"),
             [Input('submit-button-one','n_clicks'),Input('submit-button-two', 'n_clicks'),
             ])
def on_click(bt1,bt2):
    if (int(bt1) == 0 and int(bt2) == 0):
        df = pd.DataFrame.from_records(shelter.readAll({}))
        
    elif (int(bt1) > int(bt2)):
        df = pd.DataFrame(list(shelter.readAll({"animal_type":"Cat"})))
    elif (int(bt2) > int(bt1)):
        df = pd.DataFrame(list(shelter.readAll({"animal_type":"Dog"})))
        
    return df.to_dict('records')
    
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    # Code for pie chart
    dfp = pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
        figure = px.pie(dfp, names='breed', title = "Breeds")
        )
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
    Input('datatable-id', "derived_viewport_selected_rows")])
def update_map(viewData, rows):
    # Code for geolocation chart
    viewDF = pd.DataFrame.from_dict(viewData)
    dff = viewDF.loc[rows]
    return [
        d1.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=15, children=[
            d1.TileLayer(id="base-layer-id"),
            d1.Marker(position=[dff.loc[0,'location_lat'],dff.loc[0,'location_long']], children=[
                d1.Tooltip(dff['breed']),
                d1.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.loc[0,'name'])
                ])
            ])
        ])]


app